In [230]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

# Enlarging display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#variance detection
from sklearn.feature_selection import VarianceThreshold

### ML Pipeline

In [234]:
## Reading Data
df = pd.read_csv("data.csv")

## Clean the Data
df = clean_data(df)

## Drop Constant columns (0 variance)
df = remove_constant(df)

## OPTIONAL - Removing quasi-constant columns (dropping columns with 99% of similar rows)
#df = remove_quasi_constant(df,0.01)

## EXPERIMENTAL Remove features from heatmap analysis
    # OPTION 1 
drop_features = ['ROA(B) before interest and depreciation after tax', 'Net Income to Total Assets', 'Realized Sales Gross Margin', 'After-tax net Interest Rate', 'Continuous interest rate (after tax)', 'Net Value Per Share (C)', 'Per Share Net profit before tax (Yuan ¥)','Current Liability to Equity']
    # OPTION 2
#drop_features = ['Current Liabilities/Liability','Quick Assets/Current Liability']

## Filtering DF without above features
df = df.drop(drop_features,axis=1)
    
## Scale the dataset columns EXCEPT the target variable
df = scaling(df,"Bankrupt?")

## Split into TRAIN set and TEST set 
# We remove y at the very end since we will be performing upsampling
#df_train, df_test = split_data(df, test_size=0.2, random_state=42)

## Upsampling with pandas
#X_train, y_train = upsampling(df_train)
X,y = upsampling(df)

## Upsampling with SMOTE
#X_train, y_train = upsampling_smote(df_train)
#X,y = upsampling(df)

## Split X_test and y_test
#y_test = df_test['Bankrupt?']
#X_test = df_test.drop(['Bankrupt?'], axis=1)

df

Bankrupt?                                                  0
ROA(C) before interest and depreciation before interest    0
ROA(A) before interest and % after tax                     0
ROA(B) before interest and depreciation after tax          0
Operating Gross Margin                                     0
Realized Sales Gross Margin                                0
Operating Profit Rate                                      0
Pre-tax net Interest Rate                                  0
After-tax net Interest Rate                                0
Non-industry income and expenditure/revenue                0
Continuous interest rate (after tax)                       0
Operating Expense Rate                                     0
Research and development expense rate                      0
Cash flow rate                                             0
Interest-bearing debt interest rate                        0
Tax rate (A)                                               0
Net Value Per Share (B) 

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,Operating Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,Non-industry income and expenditure/revenue,Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Equity to Liability,Bankrupt?
0,-2.217909,-2.045798,-0.383334,0.016456,-0.023516,-0.087479,-0.616334,-0.750713,-0.545256,-0.151921,-0.829388,-1.279247,-1.275228,-1.794106,-0.671081,-0.025697,-0.471371,-0.025299,0.019997,-0.012031,-0.012262,-0.010299,-0.182259,-0.013721,-0.119118,-0.769297,-0.012111,-0.034237,-0.092520,-0.026226,1.750845,-1.750845,-0.133505,0.959784,0.041865,-0.471275,-1.460495,-0.332027,-0.540344,-0.045967,-0.038333,-0.661726,-0.407123,-0.155185,-0.017024,-0.237123,-0.038222,-2.393729,-1.155833,-1.520572,-0.861820,-0.020937,-0.072818,1.126267,-0.563752,-0.045371,-0.095887,-0.412896,-1.245334,0.568487,-0.094979,-1.232594,-0.043610,1.314027,-0.175421,1.299751,-0.019619,-0.685397,0.003948,-0.012111,-0.412896,0.558428,-0.257031,-0.108218,-1.247231,-0.206594,2.812493,-0.034272,-0.049491,-0.084274,-0.383465,-0.861611,0.680171,-0.060031,-0.098978,-0.622067,1
1,-0.673828,-0.311068,0.135068,0.014671,0.014799,-0.005957,-0.616334,-0.750713,-0.326632,-0.151921,-0.829388,-0.251878,-0.250429,-0.597379,-0.303533,-0.025697,-0.550058,-0.027130,0.010035,0.039473,0.039663,-0.001930,0.207732,-0.013721,0.027847,-0.143117,-0.012111,-0.034237,0.372093,-0.026226,1.075727,-1.075727,-0.132282,0.129305,-0.010932,-0.548362,-0.446793,-0.355356,-0.762702,-0.045967,-0.038333,2.220288,-0.116896,-0.357471,-0.017024,-0.277557,-0.038222,-1.067133,-1.351081,-1.558280,-0.783873,-0.020937,-0.072818,-0.670363,-0.361990,1.169887,-0.095887,-2.192039,-0.328953,-0.123797,-0.094979,-0.143474,-0.026740,-0.135126,-0.423919,1.640522,-0.010126,0.006134,0.004193,-0.012111,-2.192039,0.269947,-0.182201,-0.095129,-0.449376,-0.109511,0.527485,-0.0342

In [221]:
y

2976    1
5495    0
5197    0
4146    0
1753    1
       ..
5       1
123     0
2023    1
4918    1
948     1
Name: Bankrupt?, Length: 13198, dtype: int64

### Data Cleaning

In [21]:
def clean_data(df):
    
    # Remove space from columns names
    df =df.rename(columns=lambda x: x.strip())
    
    # Check for missing values
    print(df.isnull().sum())

    # Check for duplicates
    print(df.duplicated().sum())

    # Drop duplicates
    df.drop_duplicates(inplace=True)
    
    # Remove outliers
    # Q1 = df.quantile(0.25)
    # Q3 = df.quantile(0.75)
    # IQR = Q3 - Q1
    #df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df

In [23]:
# df = clean_data(df)

### Removing Constant

In [24]:
def remove_constant(df):
    var_thr = VarianceThreshold(threshold = 0) #Removing constant
    var_thr.fit(df)
    var_thr.get_support() #False means constant or low variance
    drop_col = [column for column in df.columns 
          if column not in df.columns[var_thr.get_support()]]
    for features in drop_col:
        print(features)
    df = df.drop(drop_col,axis=1)
    return df

In [25]:
#df = remove_constant(df)
#df

Net Income Flag


,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Net Value Per Share (C),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Per Share Net profit before tax (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Current Liability to Equity,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,1.256969e-04,0.000000e+00,0.458143,7.250725e-04,0.000000,0.147950,0.147950,0.147950,0.169141,0.311664,0.017560,0.095921,0.138736,0.022102,0.848195,0.688979,0.688979,0.217535,4.980000e+09,0.000327,0.263100,0.363725,0.002259,0.001208,0.629951,0.021266,0.207576,0.792424,0.005024,0.390284,0.006479,0.095885,0.137757,0.398036,0.086957,0.001814,0.003487,1.820926e-04,1.165007e-04,0.032903,0.034164,0.392913,0.037135,0.672775,0.166673,0.190643,0.004094,0.001997,1.473360e-04,0.147308,0.334015,0.276920,0.001036,0.676269,0.721275,0.339077,2.559237e-02,0.903225,0.002022,0.064856,7.010000e+08,6.550000e+09,0.593831,4.580000e+08,0.671568,0.424206,0.676269,0.339077,0.126549,0.637555,0.458609,0.520382,0.312905,0.118250,0,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,2.897851e-04,0.000000e+00,0.461867,6.470647e-04,0.000000,0.182251,0.182251,0.182251,0.208944,0.318137,0.021144,0.093722,0.169918,0.022080,0.848088,0.689693,0.689702,0.217620,6.110000e+09,0.000443,0.264516,0.376709,0.006016,0.004039,0.635172,0.012502,0.171176,0.828824,0.005059,0.376760,0.005835,0.093743,0.168962,0.397725,0.064468,0.001286,0.004917,9.360000e+09,7.1900

### Removing quasi-constants

In [27]:
def remove_quasi_constant(df,step):
    var_thr = VarianceThreshold(threshold = step) #Removing constant
    var_thr.fit(df)
    var_thr.get_support() #False means constant or low variance
    drop_col = [column for column in df.columns 
          if column not in df.columns[var_thr.get_support()]]
    for features in drop_col:
        print(features)
    df = df.drop(drop_col,axis=1)
    return df

In [30]:
df.shape

(6819, 32)

In [187]:
def plot_correlation_heatmap_1(df):
    # Compute the correlation matrix
    corr = df.corr()

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # Set up the matplotlib figure
    #fig, ax = plt.subplots(figsize=(30, 20))

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap='coolwarm', vmax=.8, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

    plt.title('Correlation Heatmap')
    plt.show()

In [186]:
#plot_correlation_heatmap_1(df)

In [184]:
def plot_correlation_heatmap_2(df):
    # Compute the correlation matrix
    cor_matrix = df.corr().abs()

    # Upper triangle 
    upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
    
    # Identifying columns we may want to drop due to collinearity 
    zoom = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
    df_zoom = df[zoom]

    # Set up the matplotlib figure
    #fig, ax = plt.subplots(figsize=(30, 20))

    # Draw the heatmap with the mask and correct aspect ratio
    dfCorr = df_zoom.corr().abs()
    filteredDf = dfCorr[(dfCorr >= .9)]
    # plt.figure(figsize=(30,20))
    sns.heatmap(filteredDf, annot=True, cmap="Reds")
    sns.set(font_scale=3)
    plt.title('Correlation Heatmap')
    plt.show()

In [185]:
#plot_correlation_heatmap_2(df)

### Scaling

In [83]:
def scaling(df,target):
    scaler = StandardScaler()
    # select all columns without the target
    cols_to_scale = [col for col in df.columns if col != target]
    # fit the scaler to the selected columns
    scaler.fit(df[cols_to_scale])
    scaled_cols = scaler.transform(df[cols_to_scale])
    # create a new dataframe with the scaled columns and the target variable
    scaled_df = pd.DataFrame(scaled_cols, columns=cols_to_scale)
    scaled_df[target] = df[target]
    return scaled_df

### Split into TRAIN SET and TEST SET

In [100]:
def split_data(X, test_size=0.2, random_state=42):
    X_train, X_test = train_test_split(X,test_size=test_size, random_state=random_state)
    return X_train, X_test

### Upsampling

In [131]:
#Upsampling
def upsampling(df):
    category_0 = df[df['Bankrupt?'] == 0]
    category_1 = df[df['Bankrupt?'] == 1]
    c0_len = len(category_0)
    c1_len = len(category_1)
    category_1_up = category_1.sample(c0_len, replace=True)
    print(category_1_up.shape)
    data = pd.concat([category_0, category_1_up], axis=0)
    #shuffling the data
    data = data.sample(frac=1)
    y = data['Bankrupt?']
    X = data.drop(['Bankrupt?'], axis=1)
    return X, y

In [132]:
#Upsampling with SMOTE
def upsampling_smote(df):
    y = df['Bankrupt?']
    X = df.drop(['Bankrupt?'], axis=1)
    smote = SMOTE()
    X_sm, y_sm = smote.fit_resample(X, y)
    return X_sm, y_sm

In [210]:
# Creating instances of the algorithms
logistic_regression = LogisticRegression()
decision_tree = DecisionTreeClassifier()
knn = KNeighborsClassifier()
    
# Voting Classifier
voting_clf = VotingClassifier(estimators=
                              [('lr', logistic_regression), ('dt', decision_tree), ('knn', knn)], voting='hard')

models = [logistic_regression, decision_tree, knn, voting]

# Create a DataFrame to store the results
results_df = pd.DataFrame(columns=['Model', 'Accuracy (Train)', 'Accuracy (Test)', 'Recall (Train)', 'Recall (Test)', 'Precision (Train)', 'Precision (Test)'])

# Train and test the models
for i, model in enumerate(models):
    name = type(model).__name__
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    accuracy_train = accuracy_score(y_train, y_train_pred)
    accuracy_test = accuracy_score(y_test, y_test_pred)
    recall_train = recall_score(y_train, y_train_pred)
    recall_test = recall_score(y_test, y_test_pred)
    precision_train = precision_score(y_train, y_train_pred)
    precision_test = precision_score(y_test, y_test_pred)
    results_df.loc[i] = [name, accuracy_train, accuracy_test, recall_train, recall_test, precision_train, precision_test]

# Print the results DataFrame
results_df

,Model,Accuracy (Train),Accuracy (Test),Recall (Train),Recall (Test),Precision (Train),Precision (Test)
0,LogisticRegression,0.901911,0.865836,0.920545,0.745098,0.887470,0.182692
1,DecisionTreeClassifier,1.000000,0.957478,1.000000,0.254902,1.000000,0.393939
2,KNeighborsClassifier,0.979379,0.925220,1.000000,0.647059,0.960392,0.282051
3,VotingClassifier,0.986663,0.937683,1.000000,0.666667,0.974019,0.333333


In [235]:
# create the individual classifiers
clf1 = LogisticRegression()
clf2 = DecisionTreeClassifier()
clf3 = KNeighborsClassifier()

# create the Voting Classifier
voting_clf = VotingClassifier(estimators=[('lr', clf1), ('dt', clf2), ('knn', clf3)], voting='hard')

# load your data into X and y variables
#y = df['Bankrupt?']
#X= df.drop(['Bankrupt?'], axis=1)

# define the scoring metrics to evaluate the model
scoring = ['accuracy', 'precision_macro', 'recall_macro']

# perform 5-fold cross-validation on the Voting Classifier
scores = cross_validate(voting_clf, X, y, cv=5, scoring=scoring)

# print the results for each metric across the 5 folds
print("Accuracy: %0.2f (+/- %0.2f)" % (scores['test_accuracy'].mean(), scores['test_accuracy'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores['test_precision_macro'].mean(), scores['test_precision_macro'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores['test_recall_macro'].mean(), scores['test_recall_macro'].std() * 2))

Accuracy: 0.97 (+/- 0.00)
Precision: 0.97 (+/- 0.00)
Recall: 0.97 (+/- 0.00)


In [232]:
df

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,Operating Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,Non-industry income and expenditure/revenue,Operating Expense Rate,Research and development expense rate,Cash flow rate,Interest-bearing debt interest rate,Tax rate (A),Net Value Per Share (B),Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Cash Flow Per Share,Revenue Per Share (Yuan ¥),Operating Profit Per Share (Yuan ¥),Realized Sales Gross Profit Growth Rate,Operating Profit Growth Rate,After-tax Net Profit Growth Rate,Regular Net Profit Growth Rate,Continuous Net Profit Growth Rate,Total Asset Growth Rate,Net Value Growth Rate,Total Asset Return Growth Rate Ratio,Cash Reinvestment %,Current Ratio,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Debt ratio %,Net worth/Assets,Long-term fund suitability ratio (A),Borrowing dependency,Contingent liabilities/Net worth,Operating profit/Paid-in capital,Net profit before tax/Paid-in capital,Inventory and accounts receivable/Net value,Total Asset Turnover,Accounts Receivable Turnover,Average Collection Days,Inventory Turnover Rate (times),Fixed Assets Turnover Frequency,Net Worth Turnover Rate (times),Revenue per person,Operating profit per person,Allocation rate per person,Working Capital to Total Assets,Quick Assets/Total Assets,Current Assets/Total Assets,Cash/Total Assets,Quick Assets/Current Liability,Cash/Current Liability,Current Liability to Assets,Operating Funds to Liability,Inventory/Working Capital,Inventory/Current Liability,Current Liabilities/Liability,Working Capital/Equity,Current Liabilities/Equity,Long-term Liability to Current Assets,Retained Earnings to Total Assets,Total income/Total expense,Total expense/Assets,Current Asset Turnover Rate,Quick Asset Turnover Rate,Working capitcal Turnover Rate,Cash Turnover Rate,Cash Flow to Sales,Fixed Assets to Assets,Current Liability to Liability,Equity to Long-term Liability,Cash Flow to Total Assets,Cash Flow to Liability,CFO to Assets,Cash Flow to Equity,Current Liability to Current Assets,Liability-Assets Flag,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Equity to Liability,Bankrupt?
0,-2.217909,-2.045798,-0.383334,0.016456,-0.023516,-0.087479,-0.616334,-0.750713,-0.545256,-0.151921,-0.829388,-1.279247,-1.275228,-1.794106,-0.671081,-0.025697,-0.471371,-0.025299,0.019997,-0.012031,-0.012262,-0.010299,-0.182259,-0.013721,-0.119118,-0.769297,-0.012111,-0.034237,-0.092520,-0.026226,1.750845,-1.750845,-0.133505,0.959784,0.041865,-0.471275,-1.460495,-0.332027,-0.540344,-0.045967,-0.038333,-0.661726,-0.407123,-0.155185,-0.017024,-0.237123,-0.038222,-2.393729,-1.155833,-1.520572,-0.861820,-0.020937,-0.072818,1.126267,-0.563752,-0.045371,-0.095887,-0.412896,-1.245334,0.568487,-0.094979,-1.232594,-0.043610,1.314027,-0.175421,1.299751,-0.019619,-0.685397,0.003948,-0.012111,-0.412896,0.558428,-0.257031,-0.108218,-1.247231,-0.206594,2.812493,-0.034272,-0.049491,-0.084274,-0.383465,-0.861611,0.680171,-0.060031,-0.098978,-0.622067,1
1,-0.673828,-0.311068,0.135068,0.014671,0.014799,-0.005957,-0.616334,-0.750713,-0.326632,-0.151921,-0.829388,-0.251878,-0.250429,-0.597379,-0.303533,-0.025697,-0.550058,-0.027130,0.010035,0.039473,0.039663,-0.001930,0.207732,-0.013721,0.027847,-0.143117,-0.012111,-0.034237,0.372093,-0.026226,1.075727,-1.075727,-0.132282,0.129305,-0.010932,-0.548362,-0.446793,-0.355356,-0.762702,-0.045967,-0.038333,2.220288,-0.116896,-0.357471,-0.017024,-0.277557,-0.038222,-1.067133,-1.351081,-1.558280,-0.783873,-0.020937,-0.072818,-0.670363,-0.361990,1.169887,-0.095887,-2.192039,-0.328953,-0.123797,-0.094979,-0.143474,-0.026740,-0.135126,-0.423919,1.640522,-0.010126,0.006134,0.004193,-0.012111,-2.192039,0.269947,-0.182201,-0.095129,-0.449376,-0.109511,0.527485,-0.0342